In [189]:
import os
import json
import re
import pandas as pd

In [28]:
exclude = ["francesinha.md"]
recipes = [x for x in os.listdir(".") if x not in exclude]
ing_dir = {key: {} for key in recipes}

for filename in recipes:
    if filename.startswith("."):
        continue
    else:
        print(filename)
        with open (filename, "r") as f:
            recipe = f.read()
            ingredients, directions = recipe.split("## Ingredients")[1].split("## Directions")
            ing_dir[filename]["ingredients"] = ingredients
            ing_dir[filename]["directions"] = directions

creamy-mashed-potatoes.md
red-sauce.md
tuscan-style-pork-roast.md
spicy-mayo.md
pilaf.md
italian-bread.md
pancake.md
tanzania-tea-with-milk.md
cheesy-potatoe-bake.md
aglio-e-olio.md
quarkbaellchen.md
mapo-tofu.md
chili-con-carne.md
naan-bread.md
hangover-eggs.md
baked-salmon.md
beef-jerky.md
onion-raitha.md
ravioli.md
italian-mulled-wine.md
refried-beans.md
parmesan-potatoes.md
swedish-pancakes.md
chicken-parmesan.md
omelet.md
aussie-snags.md
portuguese-steak-with-beer-sauce.md
lasagna.md
dried-tomato-plum-spread.md
shrimp-and-grits.md
frittata.md
zopf.md
asian-style-chicken-sticky-sauce.md
caesar-salad.md
miso-soup.md
pan-seared-chicken.md
soleier.md
chorizo-and-chickpea-soup.md
guacamole.md
bloody-mary-mix.md
stoemp.md
torrijas.md
hamburger-patties-all-beef.md
tomato-flavored-hamburger-macaroni.md
oatmeal-pancakes.md
cheesy-meatballs.md
loaded-mexican-rice.md
danish-pancake.md
eggs.md
chicken-tacos.md
matcha-cookies.md
pizza-sauce.md
french-crepes.md
yibin-burning-noodles.md
bread.md

In [35]:
f = open("recipes.json", "w")
f.write(json.dumps(ing_dir))
f.close()

## Cleaning Up Data

In [ ]:
def recursive_elimination(string, ls):
    """
    Recursively remove unwanted words from a string
    
    string: target from which you want to eliminate all instances of elements in ls
    ls: list of values to remove from string
    """
    for elem in ls:
        if (string.__contains__(elem)):
            string = string.replace(elem, "")
            recursive_elimination(string, ls)
            
    return string

In [202]:
# Not sure why there was empty ingredients in first place
ingredients = [list(x.values())[0].lower().strip().split("\n") for x in list(ing_dir.values()) if len(list(x.keys())) != 0]
ingredients = [val.strip("* ") for sublist in ingredients for val in sublist if val.startswith("*")]

measurements = ["teaspoon", "teaspoons", "tablespoon", "tablespoons", "lb", "kg", "oz", "ml"]

for idx, x in enumerate(ingredients):
    match = re.findall(r"~*\d+\S*\d*\b", x)

    if len(match) != 0:
        for x in match:
            ingredients[idx] = ingredients[idx].replace(x, "").strip()

    recursive_elimination(x, measurements)
            

ingredients

['potatoes',
 'milk',
 'mayonnaise',
 'cheese',
 'garlic powder',
 'strips of bacon',
 'butter',
 'green onions',
 'black pepper',
 'salt',
 'can of pre-cooked beans',
 'onion',
 'garlic cloves',
 'cumin',
 'paprika',
 'spoonfuls butter',
 'cooking oil (can be olive oil)',
 'medium-sized potatoes',
 'butter',
 'parmesan cheese',
 'rosemary; dried should be fine, but fresh is obviously better',
 'tablespoon quick-cooking tapioca',
 'teaspoon salt',
 'teaspoon pepper',
 'cup milk, scalded',
 'tablespoon butter, or margarine',
 'egg yolks, beaten until thick',
 'egg whites, beaten stiff',
 'cheddar cheese (optional for cheese omelet)',
 'chicken fillet(s); i normally use chicken breasts',
 'wheat flour',
 'paprika',
 'garlic powder',
 'herbes de provence',
 'hass avocados',
 'medium onion',
 'small tomato',
 'cilantro',
 'lime',
 'of ground beef meat',
 'of an onion',
 'of cheese; i have made with different cheese types, but usually i use',
 'parsley',
 'garlic cloves',
 'egg',
 'metric c

In [190]:
df = pd.DataFrame(ingredients)

In [193]:
pd.DataFrame.to_csv(df, "./partially_cleaned_ingredients.csv", index=False)
pd.DataFrame.to_csv(df, "./cleaned_ingredients.csv", index=False)

In [198]:
'teaspoon salt'.__contains__("teaspoon")

True

In [ ]:
def recursive_elimination(string, ls):
    for elem in ls:
        if (string.__contains__(elem)):
            string.replace(elem, "")
            recursive_elimination(string, ls)
            
    return string

measurements = ["teaspoon", "teaspoons", "tablespoon", "tablespoons", "lb", "kg", "oz", "ml"]
string = "teaspoon ml tablespoon lb"

for m in measurements:
    if string.__contains__(m):
        ingredients[idx].replace(m, "").strip()